In [1]:
import pandas as pd

In [70]:
first_line = open('../data/hollins.dat', 'r').readline().replace('\n', '')
nodes_count = int(first_line.split(' ')[0])
edges_count = int(first_line.split(' ')[1])

df = pd.read_csv('../data/hollins.dat', sep='\s+', skiprows=1, header=None, names=['NodeID', 'NodeURL'])

df_id_url = df.loc[df.NodeURL.str.contains('http')]
df_id_out_id = df.loc[~df.NodeURL.str.contains('http')]
df_id_out_id.rename(columns={'NodeURL': 'OutNodeID'}, inplace=True)

df_id_out_id['OutNodeID'] = df_id_out_id['OutNodeID'].astype(int)


/tmp/ipykernel_23758/4149064277.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_out_id.rename(columns={'NodeURL': 'OutNodeID'}, inplace=True)
/tmp/ipykernel_23758/4149064277.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id_out_id['OutNodeID'] = df_id_out_id['OutNodeID'].astype(int)


In [11]:
import pyspark
from pyspark.sql import SparkSession

session = SparkSession.builder.appName('hollins').getOrCreate()

22/05/21 20:41:19 WARN Utils: Your hostname, 192-168-0-145 resolves to a loopback address: 127.0.1.1; using 192.168.0.145 instead (on interface enp6s0)
22/05/21 20:41:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/21 20:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [85]:
df_id_out_id.sort_values(by=['NodeID']).to_csv('../data/hollins_id_out_id.csv', index=False, header=False)

In [133]:
# df_id_out_id_spark = session.createDataFrame(df_id_out_id)

df_id_out_id_ranks = df_id_out_id.copy()

# sort by node id
df_id_out_id_ranks = df_id_out_id_ranks.sort_values(by=['NodeID'])

# damping factor
d = 0.85

# initialize ranks
pr = [1/nodes_count for i in range(nodes_count+1)]
pr_2 = pr.copy()

# do iterations
for _ in range(5):
    for i in range(1, nodes_count):
        out_nodes = df_id_out_id_ranks.loc[df_id_out_id_ranks.NodeID == i, 'OutNodeID']

        pr[i] = (1-d)/nodes_count + d * (sum([pr[out_node] for out_node in out_nodes]) / out_nodes.count())

    pr_2 += pr
    pr = pr_2.copy()

/tmp/ipykernel_23758/1444487391.py:20: RuntimeWarning: invalid value encountered in long_scalars
  pr[i] = (1-d)/nodes_count + d * (sum([pr[out_node] for out_node in out_nodes]) / out_nodes.count())


In [138]:
import numpy as np
pr_copy = np.array(pr)
pr_copy[np.isnan(pr_copy)] = 0
print(pr_copy[100:200]*100)

[0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334 0.0166334
 0.016

In [115]:
# compare outputs
df_outputs_gh = pd.read_csv('../data/output.csv', header=None, names=['NodeID', 'Rank'])
df_outputs_gh.NodeID += 1

In [116]:
df_outputs_gh

,NodeID,Rank
0,1,0.000
1,2,0.021
2,3,0.000
3,4,0.000
4,5,0.000
...,...,...
6007,6008,0.000
6008,6009,0.000
6009,6010,0.000
6010,6011,0.000


In [112]:
df_outputs_networkx = pd.read_csv('../data/result_hollins_id_out_id.csv', header=None, names=['NodeID', 'Rank'])
df_outputs_networkx.sort_values(by=['NodeID'], ascending=True, inplace=True)
df_outputs_networkx

,NodeID,Rank
0,1,0.000058
1,2,0.020210
24,3,0.000113
23,4,0.000060
21,5,0.000060
...,...,...
6006,6008,0.000069
6007,6009,0.000069
6008,6010,0.000069
6010,6011,0.000069
